In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd
import polars as pl
from pathlib import Path

from linearmodels.panel import PanelOLS
from statsmodels.tsa.stattools import adfuller

OUPUT_PATH = Path("../latex/imgs/res/")
OUPUT_PATH.mkdir(parents=True, exist_ok=True)
OUPUT_TABLES_PATH = Path("../latex/tables/")
OUPUT_TABLES_PATH.mkdir(parents=True, exist_ok=True)
DATA_OUTPUT_PATH = Path("../data/results/")
DATA_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

DUOPOLY_OUPUT_PATH = Path(OUPUT_PATH) / "duopoly"
DUOPOLY_OUPUT_PATH.mkdir(parents=True, exist_ok=True)

In [2]:
df = (
    pl.read_parquet(DATA_OUTPUT_PATH / "all_experiments.parquet")
    .filter((pl.col("num_agents") == 5) & (pl.col("is_symmetric")))
    .with_columns(
        (pl.col("experiment_timestamp").rank("dense")).alias("run_id"),
        (pl.col("agent").rank("dense")).alias("firm_id"),
        pl.col("chosen_price").truediv(pl.col("alpha")).round(4).alias("price"),
    )
    .rename(
        {
            "round": "period",
            "price": "price",
            "agent_prefix_type": "prompt_prefix",
        }
    )
    .with_columns(
        # concat run_id and firm_id to create a unique identifier
        pl.concat_str(["run_id", "firm_id"], separator="_").alias("run_firm_id")
    )
    .select(["run_firm_id", "run_id", "firm_id", "period", "price", "prompt_prefix"])
    .sort(["run_id", "firm_id", "period"])
)
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix
str,u32,u32,i64,f64,str
"""1_1""",1,1,1,2.75,"""P1"""
"""1_1""",1,1,2,3.0,"""P1"""
"""1_1""",1,1,3,3.25,"""P1"""
"""1_1""",1,1,4,2.9,"""P1"""
"""1_1""",1,1,5,3.1,"""P1"""
…,…,…,…,…,…
"""42_5""",42,5,296,1.401,"""P1"""
"""42_5""",42,5,297,1.402,"""P1"""
"""42_5""",42,5,298,1.401,"""P1"""


# Stationarity check
---

- https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html

The null hypothesis of the Augmented Dickey-Fuller is that there is a unit root, with the alternative that there is no unit root. If the pvalue is above a critical size, then we cannot reject that there is a unit root.

The p-values are obtained through regression surface approximation from MacKinnon 1994, but using the updated 2010 tables. If the p-value is close to significant, then the critical values should be used to judge whether to reject the null.

The autolag option and maxlag for it are described in Greene.

In [3]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .select("price")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False               101
True                109

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0          35_4       -7.344848  1.042101e-10        True
1          28_3       -4.281334  4.784612e-04        True
2          40_1        0.583560  9.872033e-01       False
3          26_5       -4.232471  5.802920e-04        True
4           9_2       -1.542180  5.125520e-01       False
..          ...             ...           ...         ...
205        24_3       -0.293809  9.263766e-01       False
206        41_5       -1.658884  4.524115e-01       False
207        42_1       -2.356443  1.544093e-01       False
208        19_4       -5.800252  4.649006e-07        True
209        35_3       -5.332251  4.693298e-06        True

[210 rows x 4 columns]

Number of stationary series: 109
Number of non-stationary series: 101


In [4]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .with_columns(
            #   pl.col("price").diff(1).fill_null(0).alias("price_diff")
            (pl.col("price").log())
            .diff(1)
            .fill_null(0)
            .alias("price_diff")  # NOTE! This uses log differences
        )
        .filter(~pl.col("price_diff").is_in([float("inf"), float("-inf")]))
        .select("price_diff")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False                 8
True                202

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0           1_2       -7.110508  3.950368e-10        True
1          30_5      -44.666714  0.000000e+00        True
2          23_3      -10.352273  2.518631e-18        True
3          27_4       -9.924325  2.934251e-17        True
4          26_4      -11.641835  2.137738e-21        True
..          ...             ...           ...         ...
205         4_3       -9.479771  3.901131e-16        True
206         8_2      -11.375884  8.748260e-21        True
207        19_4       -5.842719  3.744694e-07        True
208         4_1      -72.148876  0.000000e+00        True
209        29_2       -7.839178  6.000908e-12        True

[210 rows x 4 columns]

Number of stationary series: 202
Number of non-stationary series: 8


We need to work with price differences.

In [5]:
df = df.with_columns(
    (pl.col("price").log())
    .diff(1)
    .over("run_firm_id")
    .fill_null(0)
    .alias("price_log_diff")  # NOTE! This uses log differences
)

df = df.with_columns(
    pl.when(pl.col("price_log_diff").is_infinite())
    .then(0)
    .otherwise(pl.col("price_log_diff"))
    .alias("price_log_diff")
)
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,2.75,"""P1""",0.0
"""1_1""",1,1,2,3.0,"""P1""",0.087011
"""1_1""",1,1,3,3.25,"""P1""",0.080043
"""1_1""",1,1,4,2.9,"""P1""",-0.113944
"""1_1""",1,1,5,3.1,"""P1""",0.066691
…,…,…,…,…,…,…
"""42_5""",42,5,296,1.401,"""P1""",-0.000714
"""42_5""",42,5,297,1.402,"""P1""",0.000714
"""42_5""",42,5,298,1.401,"""P1""",-0.000714


# Fixed effects regression (trigger strategy)
---

We are interested in the responsiveness of agents to each other since it is a feature of a reward-punishment strategy. We are interested in stickiness since it measures the persistence of such rewards and punishments.

To measure responsiveness and stickiness, we perform a linear regression with the following model:
$$p_{i,r}^t = \alpha_{i,r} + \gamma p_{i,r}^{t-1} + \delta p_{-i,r}^{t-1}+\epsilon_{i,r}^{t}$$

$$\Delta \log(p_{i,r}^t) =   \gamma \Delta \log(p_{i,r}^{t-1}) + \delta \Delta \log(p_{-i,r}^{t-1})+ \Delta \epsilon_{i,r}^{t}$$


where $p_{i,r}^t$ is the price set by the agent $i$ at period $t$ of run $r$ of the experiment, $p_{i,r}^t$ is the price set by competitors at period $t$ of run $r$ and nd $α_{i,r}$ is a firm-run fixed effect.

In [6]:
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,2.75,"""P1""",0.0
"""1_1""",1,1,2,3.0,"""P1""",0.087011
"""1_1""",1,1,3,3.25,"""P1""",0.080043
"""1_1""",1,1,4,2.9,"""P1""",-0.113944
"""1_1""",1,1,5,3.1,"""P1""",0.066691
…,…,…,…,…,…,…
"""42_5""",42,5,296,1.401,"""P1""",-0.000714
"""42_5""",42,5,297,1.402,"""P1""",0.000714
"""42_5""",42,5,298,1.401,"""P1""",-0.000714


In [7]:
df_fe = (
    df.pivot(
        values="price_log_diff",
        index=["run_id", "period", "prompt_prefix"],
        on="firm_id",
    )
    .rename(
        {
            "1": "1_log_diff",
            "2": "2_log_diff",
            "3": "3_log_diff",
            "4": "4_log_diff",
            "5": "5_log_diff",
        }
    )
    .with_columns(
        [
            pl.col("1_log_diff").shift(1).alias("1_log_diff_lag"),
            pl.col("2_log_diff").shift(1).alias("2_log_diff_lag"),
            pl.col("3_log_diff").shift(1).alias("3_log_diff_lag"),
            pl.col("4_log_diff").shift(1).alias("4_log_diff_lag"),
            pl.col("5_log_diff").shift(1).alias("5_log_diff_lag"),
        ]
    )
    .filter(pl.col('period')<100)
    # Keep only disjoint periods
    .filter(pl.col("period") % 2 == 0)
    # Alternate firms: period % 5 == 1 → firm 1, 2 → firm 2, ..., 0 → firm 5
    .with_columns(
        [
            pl.when(pl.col("period") % 5 == 1)
            .then(pl.col("1_log_diff"))
            .when(pl.col("period") % 5 == 2)
            .then(pl.col("2_log_diff"))
            .when(pl.col("period") % 5 == 3)
            .then(pl.col("3_log_diff"))
            .when(pl.col("period") % 5 == 4)
            .then(pl.col("4_log_diff"))
            .otherwise(pl.col("5_log_diff"))
            .alias("price_log_diff"),
            pl.when(pl.col("period") % 5 == 1)
            .then(pl.col("1_log_diff_lag"))
            .when(pl.col("period") % 5 == 2)
            .then(pl.col("2_log_diff_lag"))
            .when(pl.col("period") % 5 == 3)
            .then(pl.col("3_log_diff_lag"))
            .when(pl.col("period") % 5 == 4)
            .then(pl.col("4_log_diff_lag"))
            .otherwise(pl.col("5_log_diff_lag"))
            .alias("price_log_diff_lag_own"),
            pl.when(pl.col("period") % 5 == 1)
            .then(
                pl.concat_list(
                    [
                        pl.col("2_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                        pl.col("5_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .when(pl.col("period") % 5 == 2)
            .then(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                        pl.col("5_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .when(pl.col("period") % 5 == 3)
            .then(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("2_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                        pl.col("5_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .when(pl.col("period") % 5 == 4)
            .then(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("2_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("5_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .otherwise(
                pl.concat_list(
                    [
                        pl.col("1_log_diff_lag"),
                        pl.col("2_log_diff_lag"),
                        pl.col("3_log_diff_lag"),
                        pl.col("4_log_diff_lag"),
                    ]
                ).list.mean()
            )
            .alias("price_log_diff_lag_comp_avg"),
        ]
    )
    .sort(["run_id", "period", "prompt_prefix"])
    .fill_nan(0)
)

df_fe

run_id,period,prompt_prefix,1_log_diff,2_log_diff,3_log_diff,4_log_diff,5_log_diff,1_log_diff_lag,2_log_diff_lag,3_log_diff_lag,4_log_diff_lag,5_log_diff_lag,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2,"""P1""",0.087011,-0.120003,0.223144,-0.133531,0.405465,0.0,0.0,0.0,0.0,0.0,-0.120003,0.0,0.0
1,4,"""P1""",-0.113944,-0.154151,0.154151,-0.087011,-0.105361,0.080043,-0.133531,0.182322,-0.154151,-0.182322,-0.087011,-0.154151,-0.013372
1,6,"""P1""",-0.101783,-0.09531,-0.064539,-0.061875,-0.024098,0.066691,-0.087011,0.133531,-0.09531,-0.068993,-0.101783,0.066691,-0.029446
1,8,"""P1""",-0.070204,-0.117783,-0.080043,-0.021979,-0.014052,0.052186,-0.105361,-0.143101,-0.021506,0.047628,-0.080043,-0.143101,-0.006763
1,10,"""P1""",-0.054067,-0.154151,0.017094,-0.031322,-0.014528,0.035718,-0.133531,-0.033902,0.00885,-0.019048,-0.014528,-0.019048,-0.030716
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
42,90,"""P1""",0.0,-0.000825,-0.000632,0.0,0.0,0.0,-0.000824,-0.001263,-0.00068,0.0,0.0,0.0,-0.000692
42,92,"""P1""",0.0,-0.000826,-0.000633,-0.000681,0.0,0.0,-0.000825,-0.000632,-0.000681,0.0,-0.000826,-0.000825,-0.000328
42,94,"""P1""",0.0008,-0.000827,-0.000634,-0.001364,0.0,0.0,-0.000827,-0.000633,-0.000681,0.0,-0.001364,-0.000681,-0.000365


In [8]:
df_fe = df_fe.select(
    [
        "run_id",
        "period",
        "prompt_prefix",
        "price_log_diff",
        "price_log_diff_lag_own",
        "price_log_diff_lag_comp_avg",
        # "price_log_diff_lag_comp_1",
        # "price_log_diff_lag_comp_2",
    ]
)
df_fe

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",-0.120003,0.0,0.0
1,4,"""P1""",-0.087011,-0.154151,-0.013372
1,6,"""P1""",-0.101783,0.066691,-0.029446
1,8,"""P1""",-0.080043,-0.143101,-0.006763
1,10,"""P1""",-0.014528,-0.019048,-0.030716
…,…,…,…,…,…
42,90,"""P1""",0.0,0.0,-0.000692
42,92,"""P1""",-0.000826,-0.000825,-0.000328
42,94,"""P1""",-0.001364,-0.000681,-0.000365


In [9]:
df_fe["prompt_prefix"].value_counts()

prompt_prefix,count
str,u32
"""P1""",1029
"""P2""",1029


## P1vsP1

In [10]:
df_fe_p1 = df_fe.filter(pl.col("prompt_prefix") == "P1").sort(["run_id", "period"])
df_fe_p1

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",-0.120003,0.0,0.0
1,4,"""P1""",-0.087011,-0.154151,-0.013372
1,6,"""P1""",-0.101783,0.066691,-0.029446
1,8,"""P1""",-0.080043,-0.143101,-0.006763
1,10,"""P1""",-0.014528,-0.019048,-0.030716
…,…,…,…,…,…
42,90,"""P1""",0.0,0.0,-0.000692
42,92,"""P1""",-0.000826,-0.000825,-0.000328
42,94,"""P1""",-0.001364,-0.000681,-0.000365


In [11]:
df_fe_p1 = df_fe_p1.to_pandas()
df_fe_p1.set_index(["run_id", "period"], inplace=True)
df_fe_p1.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
1      2                 P1       -0.120003                0.000000   
       4                 P1       -0.087011               -0.154151   
       6                 P1       -0.101783                0.066691   
       8                 P1       -0.080043               -0.143101   
       10                P1       -0.014528               -0.019048   

               price_log_diff_lag_comp_avg  
run_id period                               
1      2                          0.000000  
       4                         -0.013372  
       6                         -0.029446  
       8                         -0.006763  
       10                        -0.030716

In [12]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p1,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0136
Estimator:                   PanelOLS   R-squared (Between):              0.0553
No. Observations:                1029   R-squared (Within):               0.0136
Date:                Wed, Jul 02 2025   R-squared (Overall):              0.0143
Time:                        18:38:26   Log-likelihood                    1593.0
Cov. Estimator:                Robust                                           
                                        F-statistic:                      6.9240
Entities:                          21   P-value                           0.0010
Avg Obs:                       49.000   Distribution:                  F(2,1006)
Min Obs:                       49.000                                           
Max Obs:                       49.000   F-statistic (robust):             0.3137
                            

## P2vsP2

In [13]:
df_fe_p2 = df_fe.filter(pl.col("prompt_prefix") == "P2").sort(["run_id", "period"])
df_fe_p2

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
10,2,"""P2""",0.223144,0.0,0.0
10,4,"""P2""",0.105361,-0.287682,-0.032609
10,6,"""P2""",-0.057158,-0.027399,-0.202983
10,8,"""P2""",-0.02353,0.047628,0.026767
10,10,"""P2""",-0.028171,0.028171,0.020849
…,…,…,…,…,…
41,90,"""P2""",-0.007968,-0.007905,-0.001992
41,92,"""P2""",-0.008163,-0.008097,-0.002008
41,94,"""P2""",0.0,0.0,-0.00405


In [14]:
df_fe_p2 = df_fe_p2.to_pandas()
df_fe_p2.set_index(["run_id", "period"], inplace=True)
df_fe_p2.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
10     2                 P2        0.223144                0.000000   
       4                 P2        0.105361               -0.287682   
       6                 P2       -0.057158               -0.027399   
       8                 P2       -0.023530                0.047628   
       10                P2       -0.028171                0.028171   

               price_log_diff_lag_comp_avg  
run_id period                               
10     2                          0.000000  
       4                         -0.032609  
       6                         -0.202983  
       8                          0.026767  
       10                         0.020849

In [15]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p2,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0290
Estimator:                   PanelOLS   R-squared (Between):             -0.0071
No. Observations:                1029   R-squared (Within):               0.0290
Date:                Wed, Jul 02 2025   R-squared (Overall):              0.0283
Time:                        18:38:26   Log-likelihood                    1014.9
Cov. Estimator:                Robust                                           
                                        F-statistic:                      15.038
Entities:                          21   P-value                           0.0000
Avg Obs:                       49.000   Distribution:                  F(2,1006)
Min Obs:                       49.000                                           
Max Obs:                       49.000   F-statistic (robust):             2.9420
                            